<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_working_calculator_inside_a_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a javascript calculator in PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [26]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We are going to create a method to add some geometric artwork to the upper right corner of a `Page`. This code is not really doing difficult things, it just deals with coordinates and math a bit. 

In [27]:
# new imports
from borb.pdf.canvas.layout.shape.connected_shape import ConnectedShape
from decimal import Decimal
from borb.pdf.canvas.color.color import HexColor, X11Color
from borb.pdf.canvas.geometry.rectangle import Rectangle
from borb.pdf.page.page_size import PageSize
from borb.pdf.page.page import Page
import typing
import random

def add_gray_artwork_to_upper_right_corner(page: Page) -> None:
  """
  This method will add a gray artwork of squares and triangles in the upper right corner
  of the given Page
  """

  # define a list of gray colors
  grays: typing.List[HexColor] = [HexColor("A9A9A9"), 
                                HexColor("D3D3D3"), 
                                HexColor("DCDCDC"), 
                                HexColor("E0E0E0"),
                                HexColor("E8E8E8"),
                                HexColor("F0F0F0")]

  # we're going to use the size of the page later on,
  # so perhaps it's a good idea to retrieve it now                                
  ps: typing.Tuple[Decimal, Decimal] = PageSize.A4_PORTRAIT.value

  # now we'll write N triangles in the upper right corner
  # we'll later fill the remaining space with squares
  N: int = 4
  M: Decimal = Decimal(32)
  for i in range(0, N):
    x: Decimal = ps[0] - N * M + i * M
    y: Decimal = ps[1] - (i+1) * M
    rg: HexColor = random.choice(grays)
    ConnectedShape(points=[(x + M,y), (x + M, y + M), (x, y + M)], stroke_color=rg, fill_color=rg).paint(page, Rectangle(x, y, M, M))

  # now we can fill up the remaining space with squares    
  for i in range(0, N-1):
    for j in range(0, N-1):
      if j > i:
        continue
      x: Decimal = ps[0] - (N-1) * M + i * M
      y: Decimal = ps[1] - (j+1) * M
      rg: HexColor = random.choice(grays)
      ConnectedShape(points=[(x, y), (x + M, y), (x + M, y + M), (x, y + M)], stroke_color=rg, fill_color=rg).paint(page, Rectangle(x, y, M, M))

Similarly, I want to add some geometric artwork to the bottom of the page to balance things out a bit. I'm going to write another separate method for that.

In [28]:
# new imports
from borb.pdf.canvas.line_art.line_art_factory import LineArtFactory

def add_colored_artwork_to_bottom_right_corner(page: Page) -> None:
  """
  This method will add a blue/purple artwork of lines and squares to the bottom right corner
  of the given Page
  """
  ps: typing.Tuple[Decimal, Decimal] = PageSize.A4_PORTRAIT.value
  
  # square
  ConnectedShape(points=[(ps[0] - 32, 40), (ps[0], 40), (ps[0], 40 + 32), (ps[0] - 32, 40 + 32)], stroke_color=HexColor("f1cd2e"), fill_color=HexColor("f1cd2e")).paint(page, Rectangle(ps[0] - 32, 40, 32, 32))
  
  # square
  ConnectedShape(points=[(ps[0] - 64, 40), (ps[0] - 32, 40), (ps[0] - 32, 40 + 32), (ps[0] - 64, 40 + 32)], stroke_color=HexColor("0b3954"), fill_color=HexColor("0b3954")).paint(page, Rectangle(ps[0] - 64, 40, 32, 32))
  
  # triangle
  ConnectedShape(points=[(ps[0] - 96, 40), (ps[0] - 64, 40), (ps[0] - 64, 40 + 32)], stroke_color=HexColor("a5ffd6"), fill_color=HexColor("a5ffd6")).paint(page, Rectangle(ps[0] - 96, 40, 32, 32))
  
  # line
  r: Rectangle = Rectangle(Decimal(0), Decimal(32), ps[0], Decimal(8))
  ConnectedShape(points=LineArtFactory.rectangle(r), stroke_color=HexColor("56cbf9"), fill_color=HexColor("56cbf9")).paint(page, r)

Now we're going to create a method that adds the image of a calculator to our `Page`. Here we are using absolute layout, since we want to make absolutely sure that our `Image` is located at the same coordinates every time (even if we were to change the text around it).

In [29]:
from borb.pdf.canvas.layout.image.image import Image
from decimal import Decimal

def add_calculator_image(page: Page):
  calculator_img = Image('https://www.shopcore.nl/pub/media/catalog/product/cache/49cebce0f131f74df9ad2e5adc64fe79/t/i/ti-1726-1.jpg',
                       width=Decimal(128),
                       height=Decimal(128))
  calculator_img.paint(page, Rectangle(Decimal(595/2 - 128/2),
                                Decimal(842/2 + 128/2),
                                Decimal(600),
                                Decimal(128)))

Next up we will be adding a lot of "buttons" (they are actually annotations with associated javascript actions). To make it a bit easier on ourselves we'll separate this logic into its own method.

In [30]:
from borb.io.read.types import Name
from borb.io.read.types import String
from borb.pdf.canvas.layout.annotation.remote_go_to_annotation import RemoteGoToAnnotation

def add_invisible_button(r: Rectangle, javascript: str):
  # the next line (commented out) adds a rectangular annotation with red border
  # this makes it a lot easier to debug the calculator
  # page.append_annotation(SquareAnnotation(r, stroke_color=HexColor("ff0000"), fill_color=None))
  page.add_annotation(RemoteGoToAnnotation(r, "https://www.borbpdf.com"))
  page[Name("Annots")][-1][Name("A")][Name("S")] = Name("JavaScript")
  page[Name("Annots")][-1][Name("A")][Name("JS")] = String(javascript)

Now we are ready to add all the buttons, and have them call our main Javascript (which will be inserted later on).

In [31]:
def add_action_annotations(page: Page):
  add_invisible_button(Rectangle(Decimal(275), Decimal(492), Decimal(13), Decimal(13)), "process_token('0')")
  add_invisible_button(Rectangle(Decimal(291), Decimal(492), Decimal(13), Decimal(13)), "process_token('.')")
  add_invisible_button(Rectangle(Decimal(307), Decimal(492), Decimal(13), Decimal(13)), "process_token('=')")

  add_invisible_button(Rectangle(Decimal(275), Decimal(507), Decimal(13), Decimal(13)), "process_token('1')")
  add_invisible_button(Rectangle(Decimal(291), Decimal(507), Decimal(13), Decimal(13)), "process_token('2')")
  add_invisible_button(Rectangle(Decimal(307), Decimal(507), Decimal(13), Decimal(13)), "process_token('3')")

  add_invisible_button(Rectangle(Decimal(275), Decimal(522), Decimal(13), Decimal(13)), "process_token('4')")
  add_invisible_button(Rectangle(Decimal(291), Decimal(522), Decimal(13), Decimal(13)), "process_token('5')")
  add_invisible_button(Rectangle(Decimal(307), Decimal(522), Decimal(13), Decimal(13)), "process_token('6')")

  add_invisible_button(Rectangle(Decimal(275), Decimal(538), Decimal(13), Decimal(13)), "process_token('7')")
  add_invisible_button(Rectangle(Decimal(291), Decimal(538), Decimal(13), Decimal(13)), "process_token('8')")
  add_invisible_button(Rectangle(Decimal(307), Decimal(538), Decimal(13), Decimal(13)), "process_token('9')")

  add_invisible_button(Rectangle(Decimal(324), Decimal(551), Decimal(13), Decimal(12)), "process_token('/')")
  add_invisible_button(Rectangle(Decimal(324), Decimal(536), Decimal(13), Decimal(13)), "process_token('x')")
  add_invisible_button(Rectangle(Decimal(324), Decimal(520), Decimal(13), Decimal(13)), "process_token('-')")
  add_invisible_button(Rectangle(Decimal(324), Decimal(497), Decimal(13), Decimal(21)), "process_token('+')")

  add_invisible_button(Rectangle(Decimal(257), Decimal(541), Decimal(13), Decimal(21)), "process_token('AC')")

This part is easy, we add document level Javascript to our PDF. This script has everything in it to make our calculator actually work.

In [32]:
from borb.io.read.types import Decimal as bDecimal
from borb.io.read.types import String
from borb.io.read.types import Stream
from borb.io.read.types import Dictionary
from borb.io.read.types import List
from borb.pdf.document.document import Document


def add_document_level_javascript(doc: Document):
  # build global_js_stream
  global_js_stream = Stream()
  global_js_stream[Name("Type")] = Name("JavaScript")
  global_js_stream[Name("DecodedBytes")] = b"""
var state = 'START';
var arg1 = 0;
var arg2 = 0;
var disp = '';
var oper = '';

function to_string(f){
	if(f > 99999999){ return '99999999'; }
	if(f < -99999999){ return '-99999999'; }
	x = f.toString();
  if(x.length > 8){ x = x.substring(0, 8);}
	return x;	
}

function is_number(token){
	return token == '0' || token == '1' || token == '2' || token == '3' || token == '4' || token == '5' || token == '6' || token == '7'  || token == '8' || token == '9';
}

function is_binary_operator(token){
	return token == '+' || token == '-' || token == 'x' || token == '/';
}

function apply_operator(a1, a2, o){
	if(o == '+'){ return a1 + a2; }
	if(o == '-'){ return a1 - a2; }
	if(o == 'x'){ return a1 * a2; }
	if(o == '/'){ 
		if(a2 == 0){
			return 0;
		}
		return a1 / a2; 
	}
}

function process_token(token){
	if(token == 'AC'){
		state = 'START';
		arg1 = 0;
		arg2 = 0;
		disp = '';
		oper = '';
    this.getField("field-000").value = disp;
		return;
	}
	if(state == 'START'){
		if(token == '.'){
			disp = '0.';
      this.getField("field-000").value = disp;
			state = 'ARG1_FLOAT';
			return;
		}
		if(is_number(token)){
			disp = token;
      this.getField("field-000").value = disp;
			state = 'ARG1'
			return;
		}
	}
	/* 
	 * ARG1
	 * arg1 is being built
	 */
	if(state == 'ARG1'){
		if(token == '.'){
			disp += '.';
      this.getField("field-000").value = disp;
			state = 'ARG1_FLOAT';
			return;
		}
		if(is_number(token)){
			disp += token;
      this.getField("field-000").value = disp;
			return;
		}
		if(is_binary_operator(token)){
			arg1 = parseFloat(disp);
			disp = '';
      this.getField("field-000").value = disp;
			oper = token;
			state = 'OPERATOR'
			return;
		}
	}
	/* 
	 * ARG1_FLOAT
	 * arg1 is being built, and a decimal point has been entered
	 */
	if(state == 'ARG1_FLOAT'){
		if(is_number(token)){
			disp += token;
      this.getField("field-000").value = disp;
			return;
		}
		if(is_binary_operator(token)){
			arg1 = parseFloat(disp);
			disp = '';
      this.getField("field-000").value = disp;
			oper = token;
			state = 'OPERATOR'
			return;
		}
	}
	/* 
	 * BINARY_OPERATOR
	 * a binary operator was entered
	 */
	if(state == 'OPERATOR'){
		if(token == '.'){
			disp = '0.';
      this.getField("field-000").value = disp;
			state = 'ARG2_FLOAT';
			return;
		}
		if(is_number(token)){
			disp = token;
      this.getField("field-000").value = disp;
			state = 'ARG2'
			return;
		}
	}
	/* 
	 * ARG2
	 * arg2 is being built
	 */
	if(state == 'ARG2'){
		if(token == '.'){
			disp += '.';
      this.getField("field-000").value = disp;
			state = 'ARG2_FLOAT';
			return;
		}
		if(is_number(token)){
			disp += token;
      this.getField("field-000").value = disp;
			return;
		}
		if(is_binary_operator(token)){
			arg1 = apply_operator(arg1, parseFloat(disp), oper);
			disp = to_string(arg1);
      this.getField("field-000").value = disp;
			oper = token;
			state = 'OPERATOR'
			return;
		}
		if(token == '='){
			arg2 = parseFloat(disp);
			disp = to_string(apply_operator(arg1, arg2, oper));
      this.getField("field-000").value = disp;
			state = 'EQUALS';
			return;
		}
	}
	if(state == 'ARG2_FLOAT'){
		if(is_number(token)){
			disp += token;
      this.getField("field-000").value = disp;
			return;
		}
		if(is_binary_operator(token)){
			arg1 = apply_operator(arg1, parseFloat(disp), oper);
			disp = to_string(arg1);
      this.getField("field-000").value = disp;
			oper = token;
			state = 'OPERATOR'
			return;
		}
		if(token == '='){
			arg2 = parseFloat(disp);
			disp = to_string(apply_operator(arg1, arg2, oper));
      this.getField("field-000").value = disp;
			state = 'EQUALS';
			return;
		}
	}	
	if(state == 'EQUALS'){
		if(token == '='){
			disp = to_string(apply_operator(parseFloat(disp), arg2, oper));
      this.getField("field-000").value = disp;
			return;
		}
		if(token == '.'){
			disp = '0.';
      this.getField("field-000").value = disp;
			state = 'ARG1_FLOAT';
			return;
		}
		if(is_number(token)){
			disp = token;
      this.getField("field-000").value = disp;
			state = 'ARG1';
			return;
		}
		if(is_binary_operator(token)){
			arg1 = parseFloat(disp);
			oper = token;
			state = 'OPERATOR';
			return;
		}
	}
}
this.getField("field-000").fillColor = color.transparent;
this.getField("field-000").textFont = "Courier";
app.runtimeHighlightColor = ["RGB", 47/255, 53/255, 51/255];
  """

  global_js_stream[Name("Filter")] = Name("FlateDecode")

  # build global js dictionary
  global_js_dictionary = Dictionary()
  global_js_dictionary[Name("S")] = Name("JavaScript")
  global_js_dictionary[Name("JS")] = global_js_stream

  # build name tree
  root = doc["XRef"]["Trailer"]["Root"]
  root[Name("Names")] = Dictionary()
  names = root["Names"]
  names[Name("JavaScript")] = Dictionary()
  names["JavaScript"][Name("Kids")] = List()
  
  # build leaf
  kids_01 = Dictionary()
  kids_01[Name("Limits")] = List()
  kids_01["Limits"].append(String("js-000"))
  kids_01["Limits"].append(String("js-000"))
  kids_01[Name("Names")] = List()
  kids_01["Names"].append(String("js-000"))
  kids_01["Names"].append(global_js_dictionary)

  names["JavaScript"]["Kids"].append(kids_01)

In order to display the result of the calculations, we need to add a `TextField` that the JavaScript can modify.

In [33]:
from borb.pdf.canvas.layout.forms.text_field import TextField

def add_display(page: Page):
  r0 = Rectangle(Decimal(264), Decimal(587), Decimal(65), Decimal(15))
  ConnectedShape(LineArtFactory.rectangle(r0), 
        stroke_color=HexColor('7e838e'), 
        fill_color=HexColor('7e838e')).paint(page, r0)

  r1 = Rectangle(Decimal(264), Decimal(587), Decimal(65), Decimal(15))
  display_field = TextField(value="", font_size=Decimal(13))
  display_field.paint(page, r1)


Now we can build our `Document`

In [34]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.geometry.rectangle import Rectangle
from borb.pdf.canvas.layout.page_layout.multi_column_layout import MultiColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.color.color import HexColor
from borb.pdf.canvas.layout.image.barcode import Barcode, BarcodeType

from decimal import Decimal
from pathlib import Path

# create Document
doc: Document = Document()

# create Page
page: Page = Page()
doc.add_page(page)

# add javascript
add_document_level_javascript(doc)

# add artwork
add_gray_artwork_to_upper_right_corner(page)
add_colored_artwork_to_bottom_right_corner(page)

# add Image
add_calculator_image(page)
add_action_annotations(page)

# add TextField
add_display(page)

# create layout
layout: PageLayout = MultiColumnLayout(page, 2)

# add first Paragraph
layout.add(Paragraph("Javascript in PDF", 
                     font="Helvetica-Bold",
                     font_size=Decimal(20),
                     font_color=HexColor('56cbf9')))

# add second paragraph
layout.add(Paragraph("""
You can cause an action to occur when a bookmark or link is clicked, or when a page is viewed. 
For example, you can use links and bookmarks to jump to different locations in a document, 
execute commands from a menu, and perform other actions. """
))

# add third Paragraph
# we are explictly adding the newlines ourselves to ensure the text
# breaks nicely around the outline of the calculator
layout.add(Paragraph("""To enhance the interactive qual-
ity of a document, you can spec-
ify actions, such as changing the 
zoom value, to occur when a page 
is opened or closed.""", respect_newlines_in_text=True))

# add fourth Paragraph
layout.add(Paragraph("Trigger Types",
                     font="Helvetica-Bold",
                     font_size=Decimal(14)))

# add fifth Paragraph
layout.add(Paragraph("Triggers determine how actions are activated in media clips, pages, and form fields. For example, you can specify a movie or sound clip to play when a page is opened or closed. The available options depend on the specified page element."))

# add sixth Paragraph
layout.add(Paragraph("Javascript",
                     font="Helvetica-Bold",
                     font_size=Decimal(14)))

# add seventh Paragraph
layout.add(Paragraph("""
The JavaScript language was developed by Netscape Communications as a means to create interactive web pages more easily. Adobe has enhanced JavaScript so that you can easily integrate this level of interactivity into your PDF documents.
You can invoke JavaScript code using actions associated with bookmarks, links, and pages. You can set Document Actions which lets you create document-level JavaScript actions that apply to the entire document."""
))

# add final Paragraph
Paragraph("With enough buttons and Javascript, you could even make a functional calculator inside a PDF!", 
          font="Courier",
          font_size=Decimal(8),
          padding_left=Decimal(5),
          border_left=True).paint(page, Rectangle(Decimal(350), Decimal(450), Decimal(200), Decimal(100)))

# add QR code
Barcode("https://www.borb-pdf.com", 
        type=BarcodeType.QR, 
        width=Decimal(64), 
        height=Decimal(64)).paint(page, Rectangle(Decimal(595 - 64 - 15), Decimal(84), Decimal(64), Decimal(64)))

# store PDF
with open(Path("output.pdf"), "wb") as pdf_file_handle:
    PDF.dumps(pdf_file_handle, doc)